# Generic Rule Induction Notebook

Continuoulsy refined.

In [218]:
#%store -z acc_list 

## README

- GLRM needs to run in proper (conda) aix360 environment
- BRCG runs with proper (conda) aix360 environment
- Use aix360i environment for RIPPER

### Configuration


In [219]:
from config import config_dict
from config import config_dict_imbalanced

# document config order
CONFIG = config_dict_imbalanced["CONFIG4"]
print('Proceed with configuration:', CONFIG["NAME"])

Proceed with configuration: miniloan


In [220]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import wittgenstein as lw
import time
import warnings
import re

from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
if CONFIG['ALGO'] == 'CORELS':
    from corels import *



# from explainer import Explainer

# TODO create reference for performance using boosted trees

# import wittgenstein as lw
# from clf_utils import make_tree_dataset, make_forest, score_forest
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder, LabelEncoder


### Data

In [221]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'], nrows = 10000)
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)

    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 10000 non-null  object 
 1   creditScore          10000 non-null  int64  
 2   income               10000 non-null  int64  
 3   loanAmount           10000 non-null  int64  
 4   monthDuration        10000 non-null  int64  
 5   rate                 10000 non-null  float64
 6   approval             10000 non-null  int64  
 7   yearlyReimbursement  10000 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 625.1+ KB


0    7810
1    2190
Name: approval, dtype: int64

In [222]:
df

,name,creditScore,income,loanAmount,monthDuration,rate,approval,yearlyReimbursement
0,John Doe,736,113243,783440,162,0.030005,0,70648
1,John Doe,527,224186,1787385,183,0.054819,0,173197
2,John Doe,460,39954,1733494,75,0.055183,0,328563
3,John Doe,751,219998,1350004,72,0.043400,0,255970
4,John Doe,684,72470,1210944,160,0.037772,0,115742
...,...,...,...,...,...,...,...,...
9995,John Doe,459,242866,196320,150,0.053151,1,21529
9996,John Doe,682,219758,695206,257,0.056538,1,56053
9997,John Doe,719,103953,500897,163,0.031124,0,45265
9998,John Doe,442,285378,1402233,295,0.033014,0,83358


In [223]:
if CONFIG['TYPE'] == 'BINARY':
    print(df[CONFIG['TARGET_LABEL']].value_counts())
elif CONFIG['TYPE'] == 'CONTINUOUS':
    df[CONFIG['TARGET_LABEL']].describe()
else:
    print('Unrecognized problem type')

0    7810
1    2190
Name: approval, dtype: int64


### Train, Test Split

In [224]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (7000, 7) (7000,)
Test: (3000, 7) (3000,)


### Reference Performance

In [225]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    print('needs prior encoding of categoricals')
    # gbr = GradientBoostingRegressor(n_estimators=500, random_state=0)
    # gbr.fit(x_train, y_train)
    # # print('Training R^2:', r2_score(yTrain, gbr.predict(dfTrain)))
    # print('Test R^2:', r2_score(y_test, gbr.predict(x_test)))

### Binarization

In [226]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=True, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=False) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()
#x_train_bin[CONFIG['EXAMPLE_FEATURE']][:10]


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 9069 to 7270
Data columns (total 55 columns):
 #   Column                                         Non-Null Count  Dtype
---  ------                                         --------------  -----
 0   (name, , )                                     7000 non-null   int64
 1   (creditScore, <=, 353.9000000000001)           7000 non-null   int32
 2   (creditScore, <=, 408.0)                       7000 non-null   int32
 3   (creditScore, <=, 461.0)                       7000 non-null   int32
 4   (creditScore, <=, 514.0)                       7000 non-null   int32
 5   (creditScore, <=, 571.0)                       7000 non-null   int32
 6   (creditScore, <=, 626.0)                       7000 non-null   int32
 7   (creditScore, <=, 678.0)                       7000 non-null   int32
 8   (creditScore, <=, 733.0)                       7000 non-null   int32
 9   (creditScore, <=, 793.0)                       7000 non-null   int32
 1

feature   name       creditScore                                            \
operation                     <=                                             
value          353.9000000000001 408.0 461.0 514.0 571.0 626.0 678.0 733.0   
9069         0                 0     1     1     1     1     1     1     1   
2603         0                 0     0     0     0     1     1     1     1   
7738         0                 0     1     1     1     1     1     1     1   
1579         0                 0     0     0     1     1     1     1     1   
5058         0                 0     1     1     1     1     1     1     1   

feature          ...                rate yearlyReimbursement                  \
operation        ...                  <=                  <=                   
value     793.0  ... 0.05704456004900808  19261.200000000004 38619.8 59424.7   
9069          1  ...                   1                   0       0       0   
2603          1  ...                   1                   0       0       0   
7738          1  ...                   1                   0       0       0   
1579          1  ...                   1                   0       1       1   
5058          1  ...                   1                   0       0       0   

feature                                                           \
operation                                                          
value     79534.6 100021.5 121469.59999999998 148177.09999999986   
9069            0        0                  0                  0   
2603            0        1                  1                  1   
7738            0        0                  0                  0   
1579            1        1                  1                  1   
5058            0        0                  1                  1   

feature                                
operation                              
value     199709.60000000003 354922.5  
9069                       0        0  
2603                       1        1  
7738                       0        0  
1579                       1        1  
5058                       1        1  

[5 rows x 55 columns]

In [227]:
x_train_bin

feature   name       creditScore                                            \
operation                     <=                                             
value          353.9000000000001 408.0 461.0 514.0 571.0 626.0 678.0 733.0   
9069         0                 0     1     1     1     1     1     1     1   
2603         0                 0     0     0     0     1     1     1     1   
7738         0                 0     1     1     1     1     1     1     1   
1579         0                 0     0     0     1     1     1     1     1   
5058         0                 0     1     1     1     1     1     1     1   
...        ...               ...   ...   ...   ...   ...   ...   ...   ...   
5734         0                 0     0     0     1     1     1     1     1   
5191         0                 0     0     0     0     1     1     1     1   
5390         0                 1     1     1     1     1     1     1     1   
860          0                 0     0     0     1     1     1     1     1   
7270         0                 0     0     0     0     0     0     0     1   

feature          ...                rate yearlyReimbursement                  \
operation        ...                  <=                  <=                   
value     793.0  ... 0.05704456004900808  19261.200000000004 38619.8 59424.7   
9069          1  ...                   1                   0       0       0   
2603          1  ...                   1                   0       0       0   
7738          1  ...                   1                   0       0       0   
1579          1  ...                   1                   0       1       1   
5058          1  ...                   1                   0       0       0   
...         ...  ...                 ...                 ...     ...     ...   
5734          1  ...                   0                   0       0       1   
5191          1  ...                   1                   1       1       1   
5390          1  ...                   1                   0       0       1   
860           1  ...                   1                   0       0       1   
7270          1  ...                   1                   0       0       0   

feature                                                           \
operation                                                          
value     79534.6 100021.5 121469.59999999998 148177.09999999986   
9069            0        0                  0                  0   
2603            0        1                  1                  1   
7738            0        0                  0                  0   
1579            1        1                  1                  1   
5058            0        0                  1                  1   
...           ...      ...                ...                ...   
5734            1        1                  1                  1   
5191            1        1                  1                  1   
5390            1        1                  1                  1   
860             1        1                  1                  1   
7270            0        0                  1                  1   

feature                                
operation                              
value     199709.60000000003 354922.5  
9069                       0        0  
2603                       1        1  
7738                       0        0  
1579                       1        1  
5058                       1        1  
...                      ...      ...  
5734                       1        1  
5191                       1        1  
5390                       1        1  
860                        1        1  
7270                       1        1  

[7000 rows x 55 columns]

### Rule Induction

In [228]:
start_time = time.time()
print('Starting training for', CONFIG['ALGO'])

if CONFIG['ALGO'] == 'BRCG':
    estimator = BooleanRuleCG() # Explainer()
    # estimator.train(x_train, y_train)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        estimator.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'RIPPER':
    estimator = Ripper()
    estimator.fit(x_train_bin, y_train, pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    estimator = lw.RIPPER()
    estimator.fit(x_train_bin, y_train,class_feat=CONFIG["TARGET_LABEL"] , pos_value=CONFIG['POS_CLASS'])
elif CONFIG['ALGO'] == 'GLRM':
    linear_model = LinearRuleRegression() # lambda0=0.0005,lambda1=0.0001
    explainer = GLRMExplainer(linear_model)
    explainer.fit(x_train_bin, y_train)
elif CONFIG['ALGO'] == 'CORELS':
    estimator = CorelsClassifier(n_iter=10000, 
                     max_card=2, # feautres per statement
                     c = 0.0001 # Higher values penalise longer rulelists
                    )
    estimator.fit(x_train_bin, y_train , prediction_name = CONFIG["TARGET_LABEL"])
    
else:
    print('Unrecognized algorithm:', CONFIG['ALGO'])

end_time = time.time()
print('Training time: ' + str(end_time - start_time))

Starting training for BRCG
Learning DNF rule with complexity parameters lambda0=0.001, lambda1=0.001
Initial LP solved
Iteration: 1, Objective: 0.1071
Iteration: 2, Objective: 0.1071
Training time: 1.8719987869262695


In [229]:
CONFIG['POS_CLASS']

False

### Evaluation

In [230]:
acc_list = []
%store -r acc_list

if CONFIG['TYPE'] == 'BINARY':
    y_pred = estimator.predict(x_test_bin)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, y_pred))
    print('Precision:', precision_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('F1', f1_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    print('ConfusionMatrix', confusion_matrix(y_test, y_pred))
    print('F-2', fbeta_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, y_pred))

    acc_list.append(recall_score(y_test, y_pred, pos_label=CONFIG['POS_CLASS']))
    %store acc_list
   
elif CONFIG['TYPE'] == 'CONTINUOUS':
    y_pred = explainer.predict(x_test_bin)
    print(f'R2 Score = {r2_score(y_test, y_pred)}')
    print(f'Explained Variance = {explained_variance_score(y_test, y_pred)}')
    print(f'Mean abs. error = {mean_absolute_error(y_test, y_pred)}')
    print(f'Max error = {max_error(y_test, y_pred)}')
    


Accuracy: 0.8933333333333333
Balanced accuracy: 0.8242054233489666
Precision: 0.9205955334987593
Recall: 0.945624468988955
F1 0.9329421626152558
ConfusionMatrix [[2226  128]
 [ 192  454]]
F-2 0.9405103937806321
Mathhews 0.6740313845720253
Stored 'acc_list' (list)


In [231]:
#XGboost for Binary Classification
# storing the results in a list
if CONFIG['TYPE'] == 'BINARY' and CONFIG['BASELINE'] == True:
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_train_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train_bin.columns.values]
    x_test_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_test_bin.columns.values]

    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train_bin, y_train)
    preds = xgb_cl.predict(x_test_bin)     
    print('Accuracy:', accuracy_score(y_test, preds))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, preds))
    print('Precision:', precision_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('F-2', fbeta_score(y_test, preds, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('Mathhews', matthews_corrcoef(y_test, preds))
   

Accuracy: 0.9783333333333334
Balanced accuracy: 0.9659764947878717
Precision: 0.9847522236340533
Recall: 0.9876805437553101
F-2 0.9870934873057655
Mathhews 0.9356535537833625


In [232]:
if CONFIG['TYPE'] == 'CONTINUOUS':
    explanation = explainer.explain()
    print(explanation)
elif CONFIG['ALGO'] == 'BRCG':
    model = estimator.explain()
    if not model['isCNF']:
        print('Number of rules:', len(model['rules']))
        print(model['rules'])
elif CONFIG['ALGO'] == 'RIPPER':
    print('Rule count: ' + str(sum([len(rules) for rules in estimator.rule_map.values()])))
    print('Rule set:')
    print(estimator.rule_list_to_pretty_string())
elif CONFIG['ALGO'] == 'CORELS':
    r_length = len(estimator.rl().rules)
    print("Rule Length:", r_length)
    for i in range(len(estimator.rl().rules[0]["antecedents"])):
        an = len(estimator.rl().rules[i]["antecedents"])
        print(f"Antecedents Length Rule {i}:" , an)
elif CONFIG['ALGO'] == 'Witt_RIPPER':
    print("Rule Length:", len(estimator.ruleset_))

# uncomment the following line for a full optimized view of the model as data frame for GLRM rules
# explanation.style

Number of rules: 1
['loanAmount <= 1011094.50 AND yearlyReimbursement <= 38619.80']
